In [1]:
import requests
from datetime import datetime
import pandas as pd
df=pd.read_csv('df10.csv')
import json
def get_codeforces_rating_history(username):
    url = f"https://codeforces.com/api/user.rating?handle={username}"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            if data["status"] == "OK":
                return data["result"]
            else:
                print(f"API Error: {data.get('comment', 'No specific error message provided')}")
        except json.JSONDecodeError:
            print("Invalid JSON response from the API")
    else:
        print(f"Failed to fetch rating history for {username}. Status code: {response.status_code}")

    return None

def days_to_cross_margin(username, target_lower, target_upper):
    rating_history = get_codeforces_rating_history(username)
    
    if rating_history is None:
        return None

    current_rating = None
    start_date = None
    end_date = None

    for entry in rating_history:
        rating = entry["newRating"]
        contest_time = entry["ratingUpdateTimeSeconds"]
        contest_date = datetime.fromtimestamp(contest_time)

        if rating <= target_lower:
            start_date = contest_date
        elif rating >= target_upper:
            end_date = contest_date
            break

    if start_date and end_date:
        days_taken = (end_date - start_date).days
        return days_taken
    else:
        print(f"{username} did not cross the rating margin within the specified range.")
        return None

def add_result_to_dataframe(df):
    df['Result'] = ""
    target_lower = 1200
    target_upper = 1400

    for index, row in df.iterrows():
        username = row['Handle']
        days_taken = days_to_cross_margin(username, target_lower, target_upper)
        
        if days_taken is not None:
            if 20 <= days_taken <= 40:
                df.at[index, 'Result'] = "Yes"
            else:
                df.at[index, 'Result'] = "No"
        else:
            df.at[index, 'Result'] = "N/A"

    return df

if __name__ == "__main__":
    
    df = add_result_to_dataframe(df)

    print(df)

Failed to fetch rating history for -WIDA-. Status code: 503
Failed to fetch rating history for Lucifer_Young. Status code: 503
Macklemore did not cross the rating margin within the specified range.
Failed to fetch rating history for Noctambulant. Status code: 503
arpan.parikh did not cross the rating margin within the specified range.
Failed to fetch rating history for chmpy. Status code: 503
Failed to fetch rating history for ganganxia. Status code: 503
Failed to fetch rating history for Areshki. Status code: 503
Failed to fetch rating history for SmilingBraniac. Status code: 503
Failed to fetch rating history for WikYH. Status code: 503
Failed to fetch rating history for ayushi.mm. Status code: 503
Failed to fetch rating history for prietukani. Status code: 503
raararaara did not cross the rating margin within the specified range.
Failed to fetch rating history for Dream_Not_Uploaded. Status code: 503
Klaus26 did not cross the rating margin within the specified range.
___Johniel did 

In [2]:
df.head()

,Handle,Max Rating,Result
0,CodeAnya,1657,No
1,Hojstyer,1721,No
2,LC_535,1746,Yes
3,Menos,1788,Yes
4,ST_ring,1657,No


In [3]:
df.to_csv('outdf10.csv', index=False)